In [62]:
#Sytem
import sys,os
import datetime
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from skopt import BayesSearchCV




print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)

----------------------------------------
Python version: 3.9.10 (main, Feb 22 2022, 13:54:07) 
[GCC 11.2.0]
pandas version: 1.3.5
matplotlib version: 3.5.1
NumPy version: 1.21.5
scikit-learn version: 1.1.1
----------------------------------------


In [63]:
Target = "Perished"
PATH_LOG = "../logs/log.csv"
ID = 12
SEED = 42

In [67]:
df_train = pd.read_csv('../data/input/train_beta.csv')
df_test = pd.read_csv('../data/input/test_beta.csv')
sample = pd.read_csv('../data/input/gender_submission.csv')
log = pd.read_csv(PATH_LOG)
data = pd.concat([df_train,df_test],sort=False)
print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    df_train.shape[0], df_train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    df_test.shape[0], df_test.shape[1]))
print('-'*25)
for i in range(0,len(data.columns),4):
    print(data.columns[i:i+4])

-------------------------
Shape of Train Data
[row :891,column :22]
-------------------------
Shape of Test Data
[row :418,column :22]
-------------------------
Index(['PassengerId', 'Perished', 'Pclass', 'Name'], dtype='object')
Index(['Sex', 'Age', 'SibSp', 'Parch'], dtype='object')
Index(['Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='object')
Index(['Title', 'FareBin', 'Fare_Code', 'Family'], dtype='object')
Index(['Age_Code', 'Embarked_Code', 'Title_Code', 'C'], dtype='object')
Index(['Q', 'S'], dtype='object')


In [68]:
drop_columns = ["Name","Ticket","Cabin","Embarked","SibSp","Parch","Fare","Fare_Code","Title","Age_Code",'FareBin',"Embarked_Code"]
dataset = data.drop(columns=drop_columns)
train = dataset[:len(df_train)]
test = dataset[len(df_train):]

In [69]:
dataset.columns

Index(['PassengerId', 'Perished', 'Pclass', 'Sex', 'Age', 'Family',
       'Title_Code', 'C', 'Q', 'S'],
      dtype='object')

In [70]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [71]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [72]:
Search = "x"
Modelname = "GBC"
CV = "x"
base_model = GradientBoostingClassifier(n_estimators = 10, random_state = SEED)
base_model.fit(X_train, y_train)
print('Train Score: {}'.format(round(base_model.score(X_train, y_train), 4)))
print(' Valid Score: {}'.format(round(base_model.score(X_valid, y_valid), 4)))
STrain = base_model.score(X_train,y_train)
SValid = base_model.score(X_valid,y_valid)

Train Score: 0.8357
 Valid Score: 0.8101


In [74]:
Modelname = "GBC"
model = GradientBoostingClassifier(learning_rate=0.01)
CV = 5
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
param_grid = {
    'n_estimators':n_estimators,
    'min_samples_split':min_samples_split,
    'min_samples_leaf':min_samples_leaf,
    'max_depth':max_depth,
    'max_features':max_features,
}
print(param_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'max_features': ['auto', 'sqrt']}


In [ ]:
# Search = "GS"
# model_cv = GridSearchCV(estimator=rfc,param_grid=param_grid,scoring='accuracy',cv=CV,verbose=2,n_jobs=-1)
# model_cv.fit(X_train,y_train)

In [51]:
Search = "RS"
model_cv = RandomizedSearchCV(estimator=model,param_distributions=param_grid,n_iter=100,cv=CV,verbose=1,random_state=SEED,n_jobs=-1)
model_cv.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=GradientBoostingClassifier(learning_rate=0.01),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=1)

In [75]:
Search = "BS"
model_cv= BayesSearchCV(estimator=model,search_spaces=param_grid,n_iter=100,cv=CV,verbose =2)
model_cv.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1600; total time=   4.4s
[CV] END max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1600; total time=   4.3s
[CV] END max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1600; total time=   4.2s
[CV] END max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1600; total time=   4.2s
[CV] END max_depth=110, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=1600; total time=   4.4s
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   2.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=600; total time=   2.1s
[CV] END max_depth=None

BayesSearchCV(cv=5, estimator=GradientBoostingClassifier(learning_rate=0.01),
              n_iter=100,
              search_spaces={'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                           100, 110, None],
                             'max_features': ['auto', 'sqrt'],
                             'min_samples_leaf': [1, 2, 4],
                             'min_samples_split': [2, 5, 10],
                             'n_estimators': [200, 400, 600, 800, 1000, 1200,
                                              1400, 1600, 1800, 2000]},
              verbose=2)

In [80]:
best_model = model_cv.best_estimator_
bestparam = str(model_cv.best_params_)
print(best_model)
bestparam = str(model_cv.best_params_)
STrain = best_model.score(X_train, y_train)
SValid  = best_model.score(X_valid, y_valid)
print('Train Score: {}'.format(STrain))
print(' Valid Score: {}'.format(SValid))

GradientBoostingClassifier(learning_rate=0.01, max_depth=50,
                           max_features='sqrt', min_samples_leaf=4,
                           min_samples_split=10, n_estimators=200)
Train Score: 0.8960674157303371
 Valid Score: 0.8212290502793296


In [84]:
Row  =  len(log)
log.loc[Row] = "x"
log.loc[Row,"Modelname"] = Modelname
log.loc[Row,"Search"] = Search
log.loc[Row,"Bestparams"] = bestparam
log.loc[Row,"CV"] = CV
log.loc[Row,"STrain"] = STrain
log.loc[Row,"SValid"] = SValid
log.loc[Row,"Age_Code_N"] = 3
use_columns = test.columns.drop(["PassengerId","Perished"])
for col in use_columns:
    log.loc[Row,col] = "o"
log.fillna("x",inplace=True)
# if PublicScore is knowed
#log.loc[Row,"SPublic"] = 0.791

In [85]:
log.tail()

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code,Vestparams,Family
41,GBC,0.8356741573033708,0.8100558659217877,o,o,o,x,x,x,o,...,x,o,x,x,x,x,x,x,x,o
42,GBC,0.8356741573033708,0.8100558659217877,o,o,o,x,x,x,o,...,x,x,x,x,x,x,x,x,x,o
43,GBC,0.896067,0.821229,o,o,o,x,x,x,o,...,o,x,BS,5,x,x,x,x,x,o
44,GBC,0.896067,0.821229,o,o,o,x,x,x,o,...,o,x,BS,5,x,x,x,x,x,o
45,GBC,0.896067,0.821229,o,o,o,x,x,x,o,...,o,x,BS,5,"OrderedDict([('max_depth', 50), ('max_features...",x,x,x,x,o


In [86]:
log.to_csv(PATH_LOG,index_label=False)
log.tail()

,Modelname,STrain,SValid,Pclass,Sex,Age,SibSp,Parch,Fare,C,...,Title_Code,Embarked_Code,Search,CV,Bestparams,Title,FareBin,Fare_Code,Vestparams,Family
41,GBC,0.8356741573033708,0.8100558659217877,o,o,o,x,x,x,o,...,x,o,x,x,x,x,x,x,x,o
42,GBC,0.8356741573033708,0.8100558659217877,o,o,o,x,x,x,o,...,x,x,x,x,x,x,x,x,x,o
43,GBC,0.896067,0.821229,o,o,o,x,x,x,o,...,o,x,BS,5,x,x,x,x,x,o
44,GBC,0.896067,0.821229,o,o,o,x,x,x,o,...,o,x,BS,5,x,x,x,x,x,o
45,GBC,0.896067,0.821229,o,o,o,x,x,x,o,...,o,x,BS,5,"OrderedDict([('max_depth', 50), ('max_features...",x,x,x,x,o
